# Selector Testing Notebook

This notebook is for experimenting with XPath/CSS selectors for ChatGPT, LinkedIn, and Twitter/X.
It launches a Chrome instance with a saved user profile and tests actions like typing, copying, and clicking.
All interactions are logged to verify selector accuracy.

In [1]:
import os
import stat
import tempfile
import time

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

try:
    import pyperclip
except ImportError:
    print("pyperclip not installed. Clipboard copying will be skipped.")
    pyperclip = None


# Chrome paths
CHROME_BINARY = r"C:\Users\abhay\Downloads\chrome-win64\chrome-win64\chrome.exe"
CHROMEDRIVER_SRC = r"C:\Users\abhay\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# User profile directory
USER_PROFILE_DIR = r"C:\Users\abhay\Desktop\bots\profiles\ubhay"

def patch_and_get_driver_path():
    """Copy chromedriver to a temp location and make it executable."""
    dst_fd, dst_path = tempfile.mkstemp(suffix=".exe")
    with open(CHROMEDRIVER_SRC, "rb") as src_f, open(dst_fd, "wb") as dst_f:
        dst_f.write(src_f.read())
    os.chmod(dst_path, stat.S_IWRITE | stat.S_IREAD | stat.S_IEXEC)
    return dst_path

def launch_browser():
    """Launch a Chrome instance with the specified user profile."""
    os.makedirs(USER_PROFILE_DIR, exist_ok=True)
    options = uc.ChromeOptions()
    options.binary_location = CHROME_BINARY
    options.add_argument(f"--user-data-dir={USER_PROFILE_DIR}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    
    driver_path = patch_and_get_driver_path()
    driver = uc.Chrome(driver_executable_path=driver_path, options=options)
    return driver

def open_tabs(driver):
    """Open ChatGPT, LinkedIn, and Twitter in separate tabs."""
    # Tab 0: ChatGPT
    driver.get("https://chatgpt.com/")
    print(f"Tab 0 opened: ChatGPT (Handle: {driver.current_window_handle})")
    time.sleep(2)
    
    # Tab 1: LinkedIn
    driver.switch_to.new_window('tab')
    driver.get("https://www.linkedin.com/feed/")
    print(f"Tab 1 opened: LinkedIn (Handle: {driver.current_window_handle})")
    time.sleep(2)
    
    # Tab 2: Twitter/X
    driver.switch_to.new_window('tab')
    driver.get("https://twitter.com/home")
    print(f"Tab 2 opened: Twitter/X (Handle: {driver.current_window_handle})")
    time.sleep(2)
    
    # Switch back to ChatGPT tab
    driver.switch_to.window(driver.window_handles[0])
    print(f"Switched back to Tab 0: ChatGPT (Handle: {driver.current_window_handle})")

# Launch Chrome and open tabs
driver = launch_browser()
open_tabs(driver)

# Keep browser open for manual inspection (comment out to auto-close)
# driver.quit()

Tab 0 opened: ChatGPT (Handle: 487628AC5B1E68E3891F130332628EA4)
Tab 1 opened: LinkedIn (Handle: DD070D7E04C195BE7839E4C938040479)
Tab 2 opened: Twitter/X (Handle: 335C55A84B58191707D815D05FD0C0D1)
Switched back to Tab 0: ChatGPT (Handle: 487628AC5B1E68E3891F130332628EA4)


In [2]:
time.sleep(4)

#chatgpt typing area - works

text_to_type = "give me long dummy para well formatted on elephants, that is properly formatted, big titles, and pointers etc. Give me a detailed, well-formatted writeup on [TOPIC], using big ALL CAPS section titles, bullet points where needed, and clear spacing. The response should be in a clean text-box format (inside triple backticks) with no HTML or XML tags—just plain, organized text, ideal for copying into notes or documents. Keep it professional but simple. No asterics, no hyphens, no ems, just bullet points starting with -, and commas, fullstops, human tone, humanized"
# The ID selector '#prompt-textarea' is generally the most robust
locator = (By.CSS_SELECTOR, "#prompt-textarea")

#even these work - locator = (By.ID, "prompt-textarea")
#and - locator = (By.XPATH, "//*[@id='prompt-textarea']")

wait = WebDriverWait(driver, 10) # Wait for a maximum of 10 seconds

try:
    # 2. Wait until the textarea is visible and ready for interaction
    text_area = wait.until(EC.element_to_be_clickable(locator))

    # 3. Click the element to ensure it has focus
    text_area.click()

    # 4. Loop through the string and send each character individually
    for char in text_to_type:
        text_area.send_keys(char)
        time.sleep(0.0001) # Adjust the delay (in seconds) between keystrokes

    print("✅ Successfully typed the full text.")

except Exception as e:
    print(f"❌ An error occurred: {e}")


✅ Successfully typed the full text.


In [ ]:
time.sleep(4)
send_button_locator = (By.ID, "composer-submit-button")

#even this works - xpath_locator = "/html/body/div[1]/div/div[1]/div[2]/main/div/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/div[3]/div/button"
# send_button_locator = (By.XPATH, xpath_locator)
try:
    # 2. Wait until the button is enabled and clickable
    send_button = wait.until(EC.element_to_be_clickable(send_button_locator))

    # 3. Click the button
    send_button.click()
    print("✅ Clicked the send button.")

except Exception as e:
    print(f"❌ Could not click the send button: {e}")

In [ ]:
time.sleep(10)
wait = WebDriverWait(driver, 10) 

# This is the specific XPath locator for your button
locator = (By.XPATH, "/html/body/div[1]/div/div[1]/div[2]/main/div/div/div[1]/div/div/div[2]/article[2]/div/div/div[3]/div/div[1]/div/button")

try:
    print("⏳ Finding the 'I prefer this response' button...")

    # 1. Wait for the button to be present and clickable
    prefer_button = wait.until(
        EC.element_to_be_clickable(locator)
    )
    
    # 2. Click the button
    prefer_button.click()
    
    print("✅ Successfully clicked the 'I prefer this response' button.")

except TimeoutException:
    print("❌ Error: Could not find a clickable button with the text 'I prefer this response' within the timeout period.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
time.sleep(4)
# This locator correctly finds all the copy buttons
locator = (By.CSS_SELECTOR, "button[aria-label='Copy']")

try:
    print("⏳ Finding all 'Copy' buttons...")

    time.sleep(4)

    # 1. Wait for and get a list of all matching buttons
    all_copy_buttons = wait.until(
        EC.presence_of_all_elements_located(locator)
    )
    print(f"✅ Found {len(all_copy_buttons)} button(s).")

    # --- THE ONLY CHANGE IS HERE ---
    # 2. Get the SECOND-TO-LAST button from the list (index -2)
    target_copy_button = all_copy_buttons[-2]
    print("🎯 Targeting the second-to-last button in the list.")

    # 3. Use a JavaScript click to ensure it works
    driver.execute_script("arguments[0].click();", target_copy_button)

    print("✅✅✅ Success! The second-to-last copy button was clicked.")

except IndexError:
    # This error will happen if fewer than two copy buttons are found on the page
    print("❌ Error: Tried to get the second-to-last button, but fewer than 2 were found.")
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [ ]:
# import pyperclip
# import time
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# # --- Assume 'driver' and 'wait' are already defined ---

# try:
#     # 1. Get the content from the system clipboard
#     clipboard_content = pyperclip.paste()
    
#     if not clipboard_content:
#         print("❌ Clipboard is empty. Nothing to paste.")
#     else:
#         print("✅ Retrieved content from clipboard. Preparing to paste line by line...")
        
#         # 2. Split the content into individual lines
#         lines = clipboard_content.split('\n')
        
#         # 3. Define the locator for the actual text area
#         locator = (By.CSS_SELECTOR, "#prompt-textarea")
        
#         # 4. Find the text area, clear it, and click it to focus
#         text_area = wait.until(EC.element_to_be_clickable(locator))
#         text_area.clear()
#         text_area.click()
        
#         # 5. Loop through the lines and type them with "Shift+Enter"
#         for i, line in enumerate(lines):
#             # Type the line of text
#             text_area.send_keys(line)
            
#             # If it's NOT the last line, press Shift+Enter
#             if i < len(lines) - 1:
#                 actions = ActionChains(driver)
#                 actions.key_down(Keys.SHIFT).send_keys(Keys.ENTER).key_up(Keys.SHIFT).perform()
#                 # A small pause can help with stability
#                 time.sleep(0.1)

#         print("✅✅✅ Success! Pasted all content line by line, preserving paragraphs.")

# except NameError:
#     print("❌ Error: 'pyperclip' is not installed or imported. Please run 'pip install pyperclip'.")
# except Exception as e:
#     print(f"❌ An error occurred during the line-by-line paste: {e}")

In [ ]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Assume 'driver' and 'wait' are already defined ---
# wait = WebDriverWait(driver, 10)

time.sleep(2) 

# The prompt to be pasted, with a hardcoded topic
text_to_type = """Generate a professional-quality, wide-angle cartoon-style image with a far-off, zoomed-out perspective. The scene should have a clean, vibrant, modern cartoon look, similar to animated illustrations used in professional presentations or business infographics, not like AI face cartoons or overly stylized filters. Avoid close-up or portrait focus, instead capture the full environment or landscape, showing people, objects, and scenery from a distance to create a sense of space and scale. The colors should be bright but balanced, with soft gradients, smooth lines, and minimal exaggerated facial features.  Think of the composition like a professional editorial illustration, suitable for a business context, with subtle cartoonish charm, clear storytelling, and no childish or comic-book effects. The style should lean towards flat design with depth (shadows, layering), giving a polished and engaging feel. Do not include close-up faces or heavily stylized characters, keep human or object elements small or medium-sized within the scene, and focus on the overall visual appeal from a distance. The topic is a team collaborating in a modern office, for now just harcode this, later on we will keep the topic seperate while genrating post and image..."""


time.sleep(2) 

# 1. Switch back to the ChatGPT tab (the first tab)
print("⏳ Switching to the ChatGPT tab...")
driver.switch_to.window(driver.window_handles[0])

# 2. Open a new chat by navigating to the base URL
print("⏳ Starting a new chat...")
driver.get("https://chatgpt.com/")

locator = (By.CSS_SELECTOR, "#prompt-textarea")

#even these work - locator = (By.ID, "prompt-textarea")
#and - locator = (By.XPATH, "//*[@id='prompt-textarea']")

wait = WebDriverWait(driver, 10) # Wait for a maximum of 10 seconds

try:
    # 2. Wait until the textarea is visible and ready for interaction
    text_area = wait.until(EC.element_to_be_clickable(locator))

    # 3. Click the element to ensure it has focus
    text_area.click()

    # 4. Loop through the string and send each character individually
    for char in text_to_type:
        text_area.send_keys(char)
        time.sleep(0.0001) # Adjust the delay (in seconds) between keystrokes

    print("✅ Successfully typed the full text.")

except Exception as e:
    print(f"❌ An error occurred: {e}")

In [ ]:
time.sleep(4)
send_button_locator = (By.ID, "composer-submit-button")

#even this works - xpath_locator = "/html/body/div[1]/div/div[1]/div[2]/main/div/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/div[3]/div/button"
# send_button_locator = (By.XPATH, xpath_locator)
try:
    # 2. Wait until the button is enabled and clickable
    send_button = wait.until(EC.element_to_be_clickable(send_button_locator))

    # 3. Click the button
    send_button.click()
    print("✅ Clicked the send button.")

except Exception as e:
    print(f"❌ Could not click the send button: {e}")

In [ ]:
import requests
import os
import time
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Assume 'driver' and 'wait' are already defined ---
# The code to submit the prompt would have just run.

try:
    # 1. Wait for 2 minutes for the image to be fully generated
    print("⏳ Waiting for 2 minutes for image generation to complete...")
    time.sleep(2)
    print("✅ Wait finished. Attempting to find and save the image.")

    # 2. Define the locator to find all generated images
    locator = (By.CSS_SELECTOR, "img[alt='Generated image']")
    
    # 3. Find all image elements and get the last one
    all_images = wait.until(EC.presence_of_all_elements_located(locator))
    if not all_images:
        raise Exception("No elements found with alt='Generated image'")
    
    image_element = all_images[-1]
    print("✅ Found the generated image element.")
    
    # 4. Get the direct URL of the image from the 'src' attribute
    image_url = image_element.get_attribute('src')
    if not image_url:
        raise Exception("Image element found, but it has no 'src' attribute.")
    
    print(f"✅ Extracted image URL: {image_url[:70]}...")
    
    # 5. Download the image using the 'requests' library
    print("⏳ Downloading image...")
    response = requests.get(image_url, stream=True)
    response.raise_for_status()
    
    # --- FIX: Combine your directory path with a unique filename ---
    
    # 6. Define the DIRECTORY where you want to save the image
    save_directory = r"C:\Users\abhay\Desktop\bots\data\screenshots"
    
    # 7. Ensure the directory exists. If not, create it.
    os.makedirs(save_directory, exist_ok=True)
    
    # 8. Create a unique FILENAME for the image
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"generated_image_{timestamp}.png"
    
    # 9. Combine the directory and filename to create the full save path
    save_path = os.path.join(save_directory, file_name)
    
    # 10. Save the image to the complete file path
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
            
    print(f"✅✅✅ Success! Image saved to: {save_path}")

except Exception as e:
    print(f"❌ An error occurred while saving the image: {e}")


In [ ]:
import time
import pyperclip
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# --- Assumes 'driver' and 'wait' are already defined from previous steps ---

def parse_connection_requests(input_string):
    """Parses the 'link--one-liner; link--one-liner;' string into a list of tuples."""
    pairs = []
    # Split by the semicolon to separate each "link--one-liner" group
    for part in input_string.strip().split(';'):
        if part.strip(): # This check ignores any empty strings from trailing semicolons
            # --- UPDATED: Use '--' as the delimiter ---
            if '--' in part:
                link, one_liner = part.split('--', 1)
                # Clean up any leading/trailing whitespace
                link = link.strip()
                one_liner = one_liner.strip()
                if link:
                    pairs.append((link, one_liner))
    return pairs

def send_linkedin_connections(driver, wait, connection_string):
    """
    Navigates to LinkedIn profiles and sends connection requests using clipboard content.
    """
    # --- Switch to Tab 1 (LinkedIn) before starting ---
    print("⏳ Switching to LinkedIn tab (Tab 1)...")
    driver.switch_to.window(driver.window_handles[1])
    
    connection_pairs = parse_connection_requests(connection_string)
    if not connection_pairs:
        print("❌ No valid 'link--one-liner' pairs found in the input string.")
        return

    for link, _ in connection_pairs: # We use _ to signify we are ignoring the one-liner
        try:
            print(f"\n--- Processing profile: {link} ---")
            # 1. Navigate to the profile link
            driver.get(link)
            time.sleep(5) # Allow time for the profile page to load fully

            # --- FINAL, MOST ROBUST LOCATOR STRATEGY ---
            # 2. Locate the first <section> inside the <main> tag. This is a very stable landmark.
            landmark_locator = (By.XPATH, "//main/section[1]")
            landmark_section = wait.until(EC.presence_of_element_located(landmark_locator))
            print("✅ Found the main profile landmark section.")
            
            connect_button_found = False
            
            # 3. First, try to find a primary "Connect" BUTTON directly within the landmark.
            try:
                # This selector specifically finds a <button> with the right aria-label.
                primary_connect_locator = (By.CSS_SELECTOR, "button[aria-label*='to connect']")
                connect_button = landmark_section.find_element(*primary_connect_locator)
                
                print("✅ Found primary 'Connect' button.")
                connect_button.click()
                connect_button_found = True

            except NoSuchElementException:
                # 4. If the button isn't found, click the "More" button within the landmark.
                print("🟡 Primary 'Connect' button not found. Clicking 'More...'")
                more_button_locator = (By.CSS_SELECTOR, "button[aria-label='More actions']")
                more_button = landmark_section.find_element(*more_button_locator)
                more_button.click()
                time.sleep(5) # Wait for dropdown to appear

                # Now, find the "Connect" DIV inside the dropdown menu.
                dropdown_connect_locator = (By.CSS_SELECTOR, "div[role='button'][aria-label*='to connect']")
                connect_button_in_dropdown = wait.until(EC.element_to_be_clickable(dropdown_connect_locator))
                
                print("✅ Found 'Connect' button in the 'More' menu.")
                connect_button_in_dropdown.click()
                connect_button_found = True

            # 5. After clicking "Connect", handle the invitation modal
            if connect_button_found:
                time.sleep(5)
                # Click "Add a note"
                add_note_button_locator = (By.CSS_SELECTOR, "button[aria-label='Add a note']")
                add_note_button = wait.until(EC.element_to_be_clickable(add_note_button_locator))
                add_note_button.click()
                time.sleep(5)

                # Get the personalized message from the clipboard
                clipboard_content = pyperclip.paste()
                if not clipboard_content:
                    print("🟡 Warning: Clipboard is empty. No note will be added.")
                
                # --- FIX: Sanitize content and type directly into the focused element ---
                # Remove non-BMP characters (like emojis) that cause the error
                sanitized_content = ''.join(c for c in clipboard_content if c <= '\uFFFF')
                
                print("✅ Sanitized clipboard content. Typing directly into focused text area...")
                # Use ActionChains to type into the active element, relying on autofocus
                ActionChains(driver).send_keys(sanitized_content).perform()
                
                print(f"✅ Pasted note from clipboard.")
                time.sleep(5)

                # Locate the final "Send" button but do not click it
                send_button_locator = (By.CSS_SELECTOR, "button[aria-label='Send now']")
                send_button = wait.until(EC.element_to_be_clickable(send_button_locator))
                
                # The click action is commented out for testing purposes
                # send_button.click() 
                
                print(f"✅ 'Send' button is ready. Request to {link} was NOT sent (for testing).")
                
                # To close the modal without sending, click the dismiss button
                close_button_locator = (By.CSS_SELECTOR, "button[aria-label='Dismiss']")
                close_button = wait.until(EC.element_to_be_clickable(close_button_locator))
                close_button.click()
                print("✅ Closed connection modal.")

                time.sleep(5) # Wait before processing the next profile

        except Exception as e:
            print(f"❌ An error occurred for profile {link}: {e}")
            continue # Move to the next profile in the list

# --- Example Usage ---
# The connection string is now updated with the '--' delimiter and corrected URL.
connection_string = "https://www.linkedin.com/in/williamhgates/--microsoft fella; https://www.linkedin.com/in/atuly/--On a mission to back 100 startups, mentor bold founders, and democratize wisdom through books.; https://www.linkedin.com/in/scarmonda/--Startups Investor, Tech Co-Founder; https://www.linkedin.com/in/harshit-madan-7ba7a615/--Building Alle;"
# The line below is now active and will run the script.
send_linkedin_connections(driver, wait, connection_string)


In [ ]:
import time
import os
import pyautogui  # <-- Import the new library
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Assumes 'driver' and 'wait' are already defined from previous steps ---

try:
    # --- STEP 1: Find the two most recent images to upload ---
    print("⏳ Finding the two most recent images from the screenshots folder...")
    save_directory = r"C:\Users\abhay\Desktop\bots\data\screenshots"
    
    # Get a list of all files in the directory with their full paths
    all_files = [os.path.join(save_directory, f) for f in os.listdir(save_directory)]
    
    # Filter for files and sort them by creation time (newest first)
    image_files = sorted([f for f in all_files if os.path.isfile(f)], key=os.path.getctime, reverse=True)
    
    upload_paths = "" # Initialize as empty
    if len(image_files) >= 2:
        # Get the paths of the two most recent files
        latest_two_images = image_files[:2]
        # Format the paths for the send_keys method (separated by a newline)
        upload_paths = "\n".join(latest_two_images)
        print(f"✅ Found the two latest images to upload:\n1: {os.path.basename(latest_two_images[0])}\n2: {os.path.basename(latest_two_images[1])}")
    else:
        print("❌ Could not find at least two images in the directory. Upload will be skipped.")

    # --- STEP 2: Upload the found images to LinkedIn ---
    if upload_paths:  # Only proceed if we found images
        # Click the VISIBLE 'Add media' button to trigger the upload element
        add_media_button_locator = (By.CSS_SELECTOR, "button[aria-label='Add media']")
        
        print("\n⏳ Finding and clicking the visible 'Add media' button...")
        add_media_button = wait.until(EC.element_to_be_clickable(add_media_button_locator))
        add_media_button.click()
        print("✅ Clicked 'Add media' button.")

        time.sleep(1)

        # Find the HIDDEN file input using the specific ID
        file_input_locator = (By.ID, "media-editor-file-selector__file-input")
        
        print("⏳ Locating the hidden file input element...")
        file_input = wait.until(EC.presence_of_element_located(file_input_locator))
        
        print(f"✅ Found hidden file input. Uploading image(s)...")
        file_input.send_keys(upload_paths)
        print("✅ Image path(s) sent to LinkedIn.")

        # Use pyautogui to send an OS-level Escape key press to close the dialog
        print("⏳ Closing OS file dialog with a global 'Escape' key press...")
        time.sleep(2) # A brief pause to ensure the dialog is the active window
        pyautogui.press('escape')
        
        # Wait for the image previews to appear
        print("⏳ Waiting for image previews to load...")
        time.sleep(10)
        
        # Click the 'Next' button
        next_button_locator = (By.CSS_SELECTOR, "button[aria-label='Next']")
        print("\n⏳ Finding and clicking the 'Next' button...")
        next_button = wait.until(EC.element_to_be_clickable(next_button_locator))
        next_button.click()
        print("✅ Clicked 'Next' button.")
        
        print("\n✅✅✅ Success! Images uploaded and advanced to the final post screen.")

except Exception as e:
    print(f"❌ An error occurred during the upload process: {e}")


In [ ]:
# This cell should be run after the text and images have been added to the post.

# This robust XPath finds the button that contains the text "Post".
# normalize-space() is used to ignore extra spaces and newlines around the text.
locator = (By.XPATH, "//button[span[normalize-space()='Post']]")

# A short wait to ensure the button is enabled after uploading
time.sleep(2)

try:
    print("⏳ Finding the final 'Post' button...")
    
    # Wait until the 'Post' button is visible and clickable
    post_button = wait.until(EC.element_to_be_clickable(locator))
    
    print("✅ Found the 'Post' button. Clicking it now...")
    
    # Click the button to publish the post
    post_button.click()
    
    print("✅✅✅ Success! Post has been published.")

except Exception as e:
    print(f"❌ An error occurred while trying to click the 'Post' button: {e}")


TRYING TWITTER #WILL DO LATER


In [ ]:
# import pyperclip
# import time
# import os
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC


# # --- Helper Function to Split Text into Tweet-Sized Chunks ---
# def split_text_into_tweets(text, limit=280):
#     """
#     Splits a long string into a list of smaller strings, each within the
#     character limit, without cutting words in half.
#     """
#     tweets = []
#     while len(text) > limit:
#         # Find the last space within the character limit
#         break_point = text.rfind(' ', 0, limit)
#         if break_point == -1:
#             # If no space is found, break at the limit (for very long words)
#             break_point = limit
        
#         # Add the chunk to the list and remove it from the original text
#         tweets.append(text[:break_point].strip())
#         text = text[break_point:].strip()
    
#     # Add the final remaining piece of text
#     if text:
#         tweets.append(text)
        
#     return tweets

# # --- Main Automation Logic ---
# try:
#     # --- 1. SETUP: Get text and find the two latest images ---
#     print("⏳ Getting content from clipboard...")
#     clipboard_content = pyperclip.paste()
    
#     print("⏳ Finding the two most recent images...")
#     save_directory = r"C:\Users\abhay\Desktop\bots\data\screenshots"
#     all_files = [os.path.join(save_directory, f) for f in os.listdir(save_directory)]
#     image_files = sorted([f for f in all_files if os.path.isfile(f)], key=os.path.getctime, reverse=True)
    
#     images_to_upload = []
#     if len(image_files) >= 2:
#         images_to_upload = image_files[:2] # Get a list of the two paths
#         print("✅ Found images to upload.")
#     else:
#         print("❌ Could not find at least two images. Skipping image upload.")

#     # Split the clipboard content into a thread
#     tweet_thread = split_text_into_tweets(clipboard_content)
#     print(f"✅ Text split into a thread of {len(tweet_thread)} tweets.")

#     # --- 2. SWITCH TO X AND PREPARE THE THREAD ---
#     print("\n⏳ Switching to X (Twitter) tab...")
#     driver.switch_to.window(driver.window_handles[2]) # Assumes X is the 3rd tab
#     time.sleep(4)

#     # --- Prepare the First Tweet ---
#     print("⏳ Preparing the first tweet...")
#     driver.get("https://x.com/compose/post")
#     time.sleep(4) # Wait for compose window to load and for focus

#     # Type directly into the focused element
#     print("⏳ Typing first tweet directly...")
#     ActionChains(driver).send_keys(tweet_thread[0]).perform()
#     time.sleep(4)

#     # Upload images to the first tweet (multi-file method)
#     if images_to_upload:
#         print("⏳ Uploading images to the first tweet...")
#         upload_paths_string = "\n".join(images_to_upload)
#         file_input_locator = (By.CSS_SELECTOR, "input[data-testid='fileInput']")
#         driver.find_element(*file_input_locator).send_keys(upload_paths_string)
#         time.sleep(10) # Wait for images to process

#     # --- Prepare the Rest of the Thread ---
#     for i in range(1, len(tweet_thread)):
#         print(f"⏳ Preparing tweet {i+1}/{len(tweet_thread)} of the thread...")
        
#         # Click the "Add post" button to extend the thread
#         add_tweet_locator = (By.CSS_SELECTOR, "button[data-testid='addButton']")
#         add_tweet_button = wait.until(EC.element_to_be_clickable(add_tweet_locator))
#         add_tweet_button.click()
#         time.sleep(4)

#         # Type directly into the newly focused reply text area
#         print(f"⏳ Typing reply tweet {i+1} directly...")
#         ActionChains(driver).send_keys(tweet_thread[i]).perform()
#         time.sleep(4)
        
#         # Upload images to the reply tweet, ONE BY ONE
#         if images_to_upload:
#             print(f"⏳ Uploading images to tweet {i+1} one by one...")
#             file_input_locator = (By.CSS_SELECTOR, "input[data-testid='fileInput']")
#             for image_path in images_to_upload:
#                 # Find all inputs and target the last one, which is for the current reply
#                 all_file_inputs = driver.find_elements(*file_input_locator)
#                 all_file_inputs[-1].send_keys(image_path)
#                 time.sleep(5) # Wait for each image to process

#     # --- FINAL STEP: Post the entire thread at once ---
#     print("\n⏳ Finding the 'Post all' button to publish the thread...")
#     post_all_button_locator = (By.CSS_SELECTOR, "button[data-testid='tweetButton']")
#     post_all_button = wait.until(EC.element_to_be_clickable(post_all_button_locator))
#     post_all_button.click()

#     print("\n✅✅✅ Success! The entire thread has been posted to X.")

# except Exception as e:
#     print(f"❌ An error occurred during the X posting process: {e}")


LINKEDIN CONNECT **********

In [3]:
import time
import pyperclip
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, InvalidSessionIdException

# --- Assumes 'driver' and 'wait' are already defined from previous steps ---

def parse_connection_requests(input_string):
    """Parses the 'link--one-liner; link--one-liner;' string into a list of tuples."""
    pairs = []
    # Split by the semicolon to separate each "link--one-liner" group
    for part in input_string.strip().split(';'):
        if part.strip(): # This check ignores any empty strings from trailing semicolons
            # --- UPDATED: Use '--' as the delimiter ---
            if '--' in part:
                link, one_liner = part.split('--', 1)
                # Clean up any leading/trailing whitespace
                link = link.strip()
                one_liner = one_liner.strip()
                if link:
                    pairs.append((link, one_liner))
    return pairs

def send_linkedin_connections(driver, wait, connection_string):
    """
    Navigates to LinkedIn profiles and sends connection requests using clipboard content.
    """
    # --- NEW: Health check to ensure the browser session is still active ---
    try:
        # Access a simple property to check if the driver is still connected.
        _ = driver.title 
    except InvalidSessionIdException:
        print("❌ CRITICAL ERROR: The browser session is invalid or has been closed.")
        print("   Please re-run the cell that launches the browser to start a new session.")
        return

    # --- Switch to Tab 1 (LinkedIn) before starting ---
    print("⏳ Switching to LinkedIn tab (Tab 1)...")
    driver.switch_to.window(driver.window_handles[1])
    
    connection_pairs = parse_connection_requests(connection_string)
    if not connection_pairs:
        print("❌ No valid 'link--one-liner' pairs found in the input string.")
        return

    for link, _ in connection_pairs: # We use _ to signify we are ignoring the one-liner
        try:
            print(f"\n--- Processing profile: {link} ---")
            # 1. Navigate to the profile link
            driver.get(link)
            time.sleep(5) # Allow time for the profile page to load fully

            # --- FINAL, MOST ROBUST LOCATOR STRATEGY ---
            # 2. Locate the first <section> inside the <main> tag. This is a very stable landmark.
            landmark_locator = (By.XPATH, "//main/section[1]")
            landmark_section = wait.until(EC.presence_of_element_located(landmark_locator))
            print("✅ Found the main profile landmark section.")
            
            connect_button_found = False
            
            # 3. First, try to find a primary "Connect" BUTTON directly within the landmark.
            try:
                primary_connect_locator = (By.CSS_SELECTOR, "button[aria-label*='to connect']")
                connect_button = landmark_section.find_element(*primary_connect_locator)
                
                print("✅ Found primary 'Connect' button. Clicking with JavaScript...")
                # --- FIX: Use a JavaScript click for reliability ---
                driver.execute_script("arguments[0].click();", connect_button)
                connect_button_found = True

            except NoSuchElementException:
                # 4. If the button isn't found, click the "More" button within the landmark.
                print("🟡 Primary 'Connect' button not found. Clicking 'More...'")
                more_button_locator = (By.CSS_SELECTOR, "button[aria-label='More actions']")
                more_button = landmark_section.find_element(*more_button_locator)
                # --- FIX: Use a JavaScript click for reliability ---
                driver.execute_script("arguments[0].click();", more_button)
                time.sleep(5) # Wait for dropdown to appear

                # Now, find the "Connect" DIV inside the dropdown menu.
                dropdown_connect_locator = (By.CSS_SELECTOR, "div[role='button'][aria-label*='to connect']")
                connect_button_in_dropdown = wait.until(EC.element_to_be_clickable(dropdown_connect_locator))
                
                print("✅ Found 'Connect' button in the 'More' menu. Clicking with JavaScript...")
                # --- FIX: Use a JavaScript click for reliability ---
                driver.execute_script("arguments[0].click();", connect_button_in_dropdown)
                connect_button_found = True

            # 5. After clicking "Connect", handle the invitation modal
            if connect_button_found:
                time.sleep(5)
                # Click "Add a note"
                add_note_button_locator = (By.CSS_SELECTOR, "button[aria-label='Add a note']")
                add_note_button = wait.until(EC.element_to_be_clickable(add_note_button_locator))
                add_note_button.click()
                time.sleep(5)

                # Get the personalized message from the clipboard
                clipboard_content = pyperclip.paste()
                if not clipboard_content:
                    print("🟡 Warning: Clipboard is empty. No note will be added.")
                
                # --- FIX: Sanitize content and type directly into the focused element ---
                # Remove non-BMP characters (like emojis) that cause the error
                sanitized_content = ''.join(c for c in clipboard_content if c <= '\uFFFF')
                
                print("✅ Sanitized clipboard content. Typing directly into focused text area...")
                # Use ActionChains to type into the active element, relying on autofocus
                ActionChains(driver).send_keys(sanitized_content).perform()
                
                print(f"✅ Pasted note from clipboard.")
                time.sleep(5)

                # Locate the final "Send" button but do not click it
                send_button_locator = (By.CSS_SELECTOR, "button[aria-label='Send now']")
                send_button = wait.until(EC.element_to_be_clickable(send_button_locator))
                
                # The click action is commented out for testing purposes
                send_button.click() 
                
                print(f"✅ 'Send' button is ready. Request to {link} was NOT sent (for testing).")
                
                # To close the modal without sending, click the dismiss button
                close_button_locator = (By.CSS_SELECTOR, "button[aria-label='Dismiss']")
                close_button = wait.until(EC.element_to_be_clickable(close_button_locator))
                close_button.click()
                print("✅ Closed connection modal.")

                time.sleep(5) # Wait before processing the next profile

        except Exception as e:
            print(f"❌ An error occurred for profile {link}: {e}")
            continue # Move to the next profile in the list

# --- Example Usage ---
# The connection string is now updated with the '--' delimiter and corrected URL.
connection_string = """https://www.linkedin.com/in/jani-pasha-b28a8023/--Dummy one-liner;
https://www.linkedin.com/in/abhishekkejriwal/--Dummy one-liner;
https://www.linkedin.com/in/utham-gowda-18652136/--Dummy one-liner;
https://www.linkedin.com/in/matt-chitharanjan-4499687/--Dummy one-liner;
https://www.linkedin.com/in/rohan-mirchandani-53321b1/--Dummy one-liner;
https://www.linkedin.com/in/sumitghosh/--Dummy one-liner;
https://www.linkedin.com/in/vaibhav-singh-5067b610/--Dummy one-liner;
https://www.linkedin.com/in/aniketdeb/--Dummy one-liner;
https://www.linkedin.com/in/chakradhar-gade-a9b3a510/--Dummy one-liner;
https://www.linkedin.com/in/rohan-nayak-9a40492b/--Dummy one-liner;
https://www.linkedin.com/in/darpansanghavi/--Dummy one-liner;
https://www.linkedin.com/in/ananthnarayanan/--Dummy one-liner;
https://www.linkedin.com/in/manishtaneja/--Dummy one-liner;
https://www.linkedin.com/in/ankit-garg-35b29011/--Dummy one-liner;
https://www.linkedin.com/in/aman-gupta-78215610/--Dummy one-liner;
https://www.linkedin.com/in/peyushbansal/--Dummy one-liner;
https://www.linkedin.com/in/amit-kumar-agarwal-5457271/--Dummy one-liner;
https://www.linkedin.com/in/nirajsingh/--Dummy one-liner;
https://www.linkedin.com/in/rajanbajaj/--Dummy one-liner;
https://www.linkedin.com/in/vineeta-singh-35920b3/--Dummy one-liner;
https://www.linkedin.com/in/rahulgarg8/--Dummy one-liner;
https://www.linkedin.com/in/asishmohapatra/--Dummy one-liner;
https://www.linkedin.com/in/varundua/--Dummy one-liner;
https://www.linkedin.com/in/kush-taneja-a56587104/--Dummy one-liner;
https://www.linkedin.com/in/kamesh-goyal-83274513/--Dummy one-liner;
https://www.linkedin.com/in/henriquedubugras/--Dummy one-liner;
https://www.linkedin.com/in/joshuareeves/--Dummy one-liner;
https://www.linkedin.com/in/parkerconrad/--Dummy one-liner;
https://www.linkedin.com/in/danielyanisse/--Dummy one-liner;
https://www.linkedin.com/in/zachperret/--Dummy one-liner;
https://www.linkedin.com/in/peterreinhardt/--Dummy one-liner;
https://www.linkedin.com/in/max-rhodes-3a1b8b1a/--Dummy one-liner;
https://www.linkedin.com/in/wadefoster/--Dummy one-liner;
https://www.linkedin.com/in/nicolasdessaigne/--Dummy one-liner;
https://www.linkedin.com/in/alexsolomon/--Dummy one-liner;
https://www.linkedin.com/in/ryanpetersen/--Dummy one-liner;
https://www.linkedin.com/in/alexandr-wang/--Dummy one-liner;
https://www.linkedin.com/in/davidhsu/--Dummy one-liner;
https://www.linkedin.com/in/cacioppo/--Dummy one-liner;
https://www.linkedin.com/in/alexbouaziz/--Dummy one-liner;
https://www.linkedin.com/in/howieliu/--Dummy one-liner;
https://www.linkedin.com/in/eglyman/--Dummy one-liner;
https://www.linkedin.com/in/grantlafontaine/--Dummy one-liner;
https://www.linkedin.com/in/dfinzer/--Dummy one-liner;
https://www.linkedin.com/in/chrisbest/--Dummy one-liner;
https://www.linkedin.com/in/jackaltman/--Dummy one-liner;
https://www.linkedin.com/in/jondahl/--Dummy one-liner;
https://www.linkedin.com/in/mathildecollin/--Dummy one-liner;
https://www.linkedin.com/in/vladmagdalin/--Dummy one-liner;
https://www.linkedin.com/in/amjadmasad/--Dummy one-liner;
"""
send_linkedin_connections(driver, wait, connection_string)

⏳ Switching to LinkedIn tab (Tab 1)...

--- Processing profile: https://www.linkedin.com/in/jani-pasha-b28a8023/ ---
❌ An error occurred for profile https://www.linkedin.com/in/jani-pasha-b28a8023/: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff78937cda5+78885]
	GetHandleVerifier [0x0x7ff78937ce00+78976]
	(No symbol) [0x0x7ff789139bca]
	(No symbol) [0x0x7ff789190766]
	(No symbol) [0x0x7ff789190a1c]
	(No symbol) [0x0x7ff7891e4467]
	(No symbol) [0x0x7ff7891b8bcf]
	(No symbol) [0x0x7ff7891e122f]
	(No symbol) [0x0x7ff7891b8963]
	(No symbol) [0x0x7ff7891816b1]
	(No symbol) [0x0x7ff789182443]
	GetHandleVerifier [0x0x7ff789654eed+3061101]
	GetHandleVerifier [0x0x7ff78964f33d+3037629]
	GetHandleVerifier [0x0x7ff78966e592+3165202]
	GetHandleVerifier [0x0x7ff78939730e+186766]
	GetHandleVerifier [0x0x7ff78939eb3f+217535]
	GetHandleVerifier [0x0x7ff7893859b4+114740]
	GetHandleVerifier [0x0x7ff789385b69+115177]
	GetHandleVerifier [0x0x7ff78936c368+10728]
	BaseThreadInitThunk [0x0x7fff976d259d+29